In [1]:
# TDO COMBINE TRAIN HISTORY DATA TO ONE FILE PER DAY
# TODO CALC TRAVALED PASSENGERN FROM TO FOR EACH DAY FOR EVERY STATION TO ONE! NEXT STATION

In [16]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime
import json


cwd = os.getcwd()
dep_table_path = os.path.join(cwd, 'generated/departure_tables_raw')
dep_table_path

'/home/jovyan/work/generated/departure_tables_raw'

In [17]:
raw_departure_tables = []
departure_tables_dates = []
departure_tables_dates_ts = []
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( dep_table_path)] #list comprehension version.

for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        if '_' in filename and len(filename.split('_')) == 4:
            # THE GOAL IS TO COMBINE ALL TRAIN DEPARTURES FROM THE SAME DAY
            # SO THE FILE TIMESTAMP IN ITS NAME CORRESPDS TO THE DATE OF RECORD = TRAIN DEPARTURES
            # BY ADDING THE NEW DATE TO THE PD DF ITS IS POSSIBLE TO SORT FOR DATES LATER ON
            timestamp = int(filename.split('_')[3])
            date = datetime.fromtimestamp(timestamp)
            # THIS WILL BE ADDED TO THE DF AS NEW COLUMN AFTER LOADING
            combined_day = str(date.day).zfill(2) + str(date.month).zfill(2) + str(date.year).zfill(2)
            departure_tables_dates.append(combined_day)
            departure_tables_dates_ts.append(timestamp)
            # LOAD FILE TO DF
            abs_path = os.path.join(dep_table_path,f)
            df = pd.read_csv(abs_path,sep=';',encoding="utf-8",dtype = str)
            
            # ADD NEW COLUMNS
            df['date'] = pd.Series([date for x in range(len(df.index))])
            df['combined_date'] = pd.Series([combined_day for x in range(len(df.index))])
            
            raw_departure_tables.append(df)


            
# CREATE DATASET FROM DATE VALUES
df_departure_tables_dates = pd.DataFrame(data={'recorded_days':departure_tables_dates})
            
# AFER LOADING ALL RAW FILE IN CREATE ONE GIANT DATASET
departures_combined = pd.concat(raw_departure_tables)
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,2021-12-03T04:45:00.000Z,Bayreuth Hbf,RE,RE 38,38,59318,8000028,"8000028%2021-12-03T04:45:00.000Z%0,8000974,800...",602,2021-12-03 04:23:22,03122021
1,2021-12-03T04:51:00.000Z,Bayreuth Hbf,RE,RE 30,30,3450,8000028,"8000028%2021-12-03T04:51:00.000Z%,8001348,8000...",602,2021-12-03 04:23:22,03122021
2,2021-12-03T05:24:00.000Z,Bayreuth Hbf,RE,RE 30,30,3402,8000028,"8000028%2021-12-03T05:25:00.000Z%1,8004759,800...",602,2021-12-03 04:23:22,03122021
3,2021-12-03T06:01:00.000Z,Bayreuth Hbf,RE,RE 32,32,59300,8000028,"8000028%2021-12-03T06:01:00.000Z%,8005895,8000...",602,2021-12-03 04:23:22,03122021
4,2021-12-03T06:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3454,8000028,"8000028%2021-12-03T06:03:00.000Z%,8004759,8004...",602,2021-12-03 04:23:22,03122021


In [18]:
# REMOVE DUPLICATE TRAINS TRAIN_ID AND START_STATION
departures_combined = departures_combined.drop_duplicates(['number','current_station_name','combined_date'],keep= 'last')
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
1195,2021-12-03T23:36:00.000Z,Lang Göns,RB,RB 40,40,15137,8003520,"8003520%2021-12-03T23:36:00.000Z%,8003262,8001...",426,2021-12-02 23:47:48,02122021
1755,2021-12-03T23:18:00.000Z,Neuhof(Kr Fulda),RE,RE 50,50,4541,8004295,"8004295%2021-12-03T23:18:00.000Z%,8002010,8000...",602,2021-12-02 23:47:48,02122021
1878,2021-12-03T23:33:00.000Z,Münster-Häger,RB,RB 64,64,20235,8004426,"8004426%2021-12-03T23:33:00.000Z%,8004173,8000...",426,2021-12-02 23:47:48,02122021
1993,2021-12-03T23:42:00.000Z,Oberbrechen,RB,RB 22,22,15293,8004518,"8004518%2021-12-03T23:42:00.000Z%,8004409,8001...",426,2021-12-02 23:47:48,02122021
2530,2021-12-03T23:39:00.000Z,Pönitz(Holst),RB,RB 84,84,11188,8004848,"8004848%2021-12-03T23:39:00.000Z%,8001941,8003...",426,2021-12-02 23:47:48,02122021


In [19]:
# LETS TEST THE DATA
# GET ALL TRAIN DEPARTURES FROM Bayreuth Hbf [8000028] @ 29.11.2021
# RV + FV + DB ONLY
tmp = departures_combined.loc[departures_combined['current_station'] == str(8000028)]
tmp = tmp.loc[tmp['combined_date'] == departure_tables_dates[0]]
tmp = tmp.reset_index()
tmp.head(100)

,index,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date
0,5,2021-12-03T16:15:00.000Z,Bayreuth Hbf,RE,RE 30,30,3414,8000028,"8000028%2021-12-03T16:15:00.000Z%,8001348,8000...",602,2021-12-03 14:48:41,03122021
1,2,2021-12-03T04:51:00.000Z,Bayreuth Hbf,RE,RE 30,30,3450,8000028,"8000028%2021-12-03T04:51:00.000Z%,8001348,8000...",602,2021-12-03 00:38:20,03122021
2,3,2021-12-03T05:24:00.000Z,Bayreuth Hbf,RE,RE 30,30,3402,8000028,"8000028%2021-12-03T05:24:00.000Z%,8004759,8004...",602,2021-12-03 00:38:20,03122021
3,5,2021-12-03T06:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3454,8000028,"8000028%2021-12-03T06:03:00.000Z%,8004759,8004...",602,2021-12-03 00:38:20,03122021
4,7,2021-12-03T06:32:00.000Z,Bayreuth Hbf,RE,RE 30,30,3081,8000028,"8000028%2021-12-03T06:32:00.000Z%,8004126,8002...",602,2021-12-03 00:38:20,03122021
5,12,2021-12-03T08:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3456,8000028,"8000028%2021-12-03T08:03:00.000Z%,8004759,8002...",602,2021-12-03 00:38:20,03122021
6,4,2021-12-03T10:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3458,8000028,"8000028%2021-12-03T10:03:00.000Z%,8004759,8002...",602,2021-12-03 09:07:21,03122021
7,12,2021-12-03T15:29:00.000Z,Bayreuth Hbf,RE,RE 30,30,3088,8000028,"8000028%2021-12-03T15:29:00.000Z%,8004759,8004...",602,2021-12-03 12:09:55,03122021
8,13,2021-12-03T16:02:00.000Z,Bayreuth Hbf,RE,RE 38,38,59310,8000028,"8000028%2021-12-03T16:02:00.000Z%,8005895,8000...",602,2021-12-03 12:09:55,03122021
9,14,2021-12-03T16:03:00.000Z,Bayreuth Hbf,RE,RE 32,32,3464,8000028,"8000028%2021-12-03T16:03:00.000Z%,8004759,8002...",602,2021-12-03 12:09:55,03122021


In [20]:
# COUNT THE UNIQUE TRAINS
#THE TRAIN_ID IS UNIQUE FOR EACH TRAIN AND IDENTIFIES THE TRAIN OVER ITS COMPLETE COURSE
# ONE UNIQUE ID = ONE ZUGLAUF
# FROM START TO END-STATION
tmp = departures_combined.loc[departures_combined['combined_date'] == departure_tables_dates[0]]
tmp['number'].nunique()

9153

In [21]:
# REMOVE DUPLICATES IN THE DATES
df_departure_tables_dates = df_departure_tables_dates.drop_duplicates()

In [22]:
# SAVE RESULTS AS CSV
departures_combined.to_csv("./generated/4_departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/4_departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)

In [37]:
# SAVE RESULT AS JSON FOR WEBSERVER
df_departure_tables_dates

,recorded_days
0,03122021
1,05122021
3,02122021
4,29112021
5,30112021
6,04122021
15,06122021
18,01122021


In [38]:
jdict = {
    'recorded_dates': df_departure_tables_dates['recorded_days'].values.tolist()
}
jdict

{'recorded_dates': ['03122021',
  '05122021',
  '02122021',
  '29112021',
  '30112021',
  '04122021',
  '06122021',
  '01122021']}

In [39]:
jdict
jdict_string = json.dumps(jdict)
with open("./generated/4_departure_table_day_list.json", "w") as outfile:
    json.dump(jdict_string, outfile)